In [ ]:
"""This code calculates from their definition the non-zero Walsh angles of the diagonal unitary U(t).
Then, it also calculates the number of gates (rotations and cnots) necessary to implement U(t)."""

In [ ]:
import numpy as np

In [ ]:
# Choose any dimension d
d = 16

In [ ]:
# Number of qubits
q = 2 * int(np.ceil(np.log(d) / np.log(2)))

# Discretize the interval x = [0,1) into N = 2^q parts
x = [i / 2**q for i in range(2**q)]

In [ ]:
def binary_expansion(integer):
    # Returns the binary expansion of an integer.
    binary_representation = bin(integer)[2:]
    return [int(bit) for bit in binary_representation][::-1]

def dyadic_expansion(number):
    # Returns the dyadic expansion of an integer
    coefficients = []
    while number > 0:
        number *= 2
        whole_part = int(number)
        coefficients.append(whole_part)
        number -= whole_part
    return coefficients

In [ ]:
# Initialize matrices
binj = np.zeros((2**q, q), dtype=int)
dyadj = np.zeros((2**q, q), dtype=int)
wjk = np.zeros((2**q, 2**q), dtype=int)

# Populate binj and dyadj matrices
for i in range(2**q):
    bin_expansion_i = binary_expansion(i)[:q] + [0] * (q - len(binary_expansion(i)))
    dyad_expansion_i = dyadic_expansion(x[i])[:q] + [0] * (q - len(dyadic_expansion(x[i])))
    
    binj[i, :] = bin_expansion_i
    dyadj[i, :] = dyad_expansion_i

# Calculate the Walsh matrix wjk
for j in range(2**q):
    for k in range(2**q):
        wjk[j, k] = (-1)**sum(binary * dyadic for binary, dyadic in zip(binary_expansion(j), dyadic_expansion(x[k])))

In [ ]:
# Calculate vector f, specific to the diagonal unitary U(t)
fk = np.zeros(2**q, dtype=int)
for l in range(1, int(2**(q/2)) + 1):
    fk[(l-1)*int(2**(q/2)) : l*int(2**(q/2))] = [l*p for p in range(1, int(2**(q/2))+1)]

# Compute Walsh angles a_j
a_j = np.dot(wjk, fk)

# Identify and count non-zero indices
non_zero_indices = [i for i, value in enumerate(a_j) if value != 0]
print("Indices with non-zero Walsh angles:", non_zero_indices)

# Show the angles a_j, excluding a_{0}
print("Non-zero Walsh angles:")
for i in non_zero_indices:
    if i == 0:  # Skip i = 0
        continue
    print(f'a_{i} =', np.array(a_j[i]))

In [ ]:
# Function to calculate Hamming weight of an integer j
def hamming_weight(j):
    return bin(j).count('1')
    
# Number of total rotations
rot = len(non_zero_indices) - 1
# Calculate total gates
total_gates = sum(2 * (hamming_weight(non_zero_indices[i]) - 1) for i in range(1, rot + 1)) + rot

# Show the total number of non-zero indices and the total number of gates necessary to implement U(t)
print(f"Number of non-zero indices: {rot}")
print(f"Total number of gates: {total_gates}")